# Домашняя работа 6. Классификация.

1. Скачайте этот ноутбук к себе.
2. Заполните пропущенные ячейки, отвечая на заданные вопросы. Там должен быть код! (если не сказано обратное)
3. Сохраните результат в своём гитхаб репозитории.
4. Пришлите на почту ml-teachers ссылку на этот файл в вашем репозитории.
5. В теме письма указать `ML Homework #6: Name Surname`. Если тема будет неправильная, то задание не будет проверено.

**Дедлайн**: 4.12.2017 01:00 OMSK

    После дедлайна можно сдать, но вы получите штраф по баллам в рейтинге.

**Рассылка решений**: 9.12.2017 17:00 OMSK

    После рассылки задания совсем не имеют веса.

### Полезная литература

- [ODS - классификация](https://habrahabr.ru/company/ods/blog/322534/)
- [Коллаборативная фильтрация - рекомендательная система](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F)
- [Метрики расстояний - косинусная, минковского, проклятье размерности ](http://www.machinelearning.ru/wiki/images/9/9a/Sem1_knn.pdf)
- [Offtop: метрики качества рекомендаьельных систем](https://habrahabr.ru/company/dca/blog/281066/)
- [Байесовский классификатор](http://www.machinelearning.ru/wiki/index.php?title=%D0%91%D0%B0%D0%B9%D0%B5%D1%81%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80)
- [Интересная статья про спам/не спам](https://habrahabr.ru/post/252265/)

Если вам понравилась задача распознавания текстов, то вот подборочка:

- [библиотека Gensim](https://radimrehurek.com/gensim/)
- [библиотека nltk](http://www.nltk.org/)
- [Лекция про word2vec 🔥](https://www.youtube.com/watch?v=oBb9aFmp0Hs)
- [Лекция Векторные представления слов и документов 🔥](https://www.youtube.com/watch?v=hiDBnEyoZS4)

В этой практике мы будем использовать алгоритмы k ближайших соседей и наивный байес для задач классификации и не только. 

## kNN - ближайшие соседи

Мы уже видели с вами на практиках и лекциях как применять алгоритм ближайших соседей. Мы даже видели с вами на примере классификации sms-спама, что этот метрический алгоритм не всегда справляется с многомерными данными. Поэтому в домашнем задании мы с вами сконцентриуемся на другой задаче, котоую можно решить этим алгоритмом: вы построите рекомендательную систему.

### Рекомендательная система

Будем использовать известный нам [датасет с оценками фильмов](http://www.cs.umn.edu/GroupLens). Только в этот раз мы не будем кластеризовывать жанры, а будем рекомендовать похожие фильмы, основываясь на оценках других пользователей. То есть мы будем по факту предсказывать предпочтение нового пользователя на основе других.

Для рекомендательной системы в датасете в объектами будут пользователи, а признаками - оценки фильма данным пользователем.

![](https://i.imgur.com/hn3xjZl.png)

Мы будем использовать метрический алгоритм для поиска ближайших соседей, тем самым рекомендовать фильмы близкие к данному.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
movies = pd.read_csv('data/imdb_movies.csv.gz', index_col='movieId')
ratings = pd.read_csv('data/imdb_ratings.csv.gz')

print(movies.shape)
print(ratings.shape)

(9125, 2)
(100004, 4)


In [3]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### 1.1 Поиск популярных фильмов (3 балла)

Среди всех фильмов есть не популярные, для которых выставлена 1-2 оценки. Такого количества данных не достаточно, поэтому нужно срезать много фильмов с маленьким числом отзывов.

Посчтитайте количество оценок для каждого фильма.

In [5]:
moviesRating = ratings.groupby(['movieId']).count()['rating']
moviesRating

movieId
1         247
2         107
3          59
4          13
5          56
6         104
7          53
8           5
9          20
10        122
11         82
12         18
13          8
14         31
15         11
16         88
17         86
18         26
19         92
20         13
21         95
22         38
23         22
24         34
25        101
26          5
27          7
28         18
29         40
30         10
         ... 
158238      2
158314      1
158528      1
158956      1
159093      2
159462      1
159690      1
159755      1
159858      2
159972      1
160080      1
160271      1
160438      2
160440      1
160563      2
160565      1
160567      1
160590      1
160656      1
160718      1
161084      1
161155      1
161594      1
161830      1
161918      1
161944      1
162376      1
162542      1
162672      1
163949      1
Name: rating, Length: 9066, dtype: int64

Выведите сводную статистику по количеству оценок фильмов. (общее число фильмов, среднее количество оценок, дисперсию, квантили)

In [6]:
moviesRating.describe()

count    9066.000000
mean       11.030664
std        24.050800
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       341.000000
Name: rating, dtype: float64

Как мы видим `75%` квантиль показывет, что `75%` фильмов имело очень мало оценок. Нам столько будет недостаточно. Посчтитайте квантили в диапазоне от `75%` до `100%` с шагом в `1%`. Выберите тот квантиль, при котором количество оценок равно 20. Напишите какой это квантиль. Напишите сколько всего фильмов останется, если отбросить все фильмы у которых рейтинг меньше этого квантиля.

In [7]:
i = .75
while (i < 1):
    if (moviesRating.quantile(i) == 20):
        break
    i+=.01
print(str(int(i*100)) + "%")

moviesRatingSorted = moviesRating.where(moviesRating > moviesRating.quantile(i)).dropna()
moviesRatingSorted

86%


movieId
1         247.0
2         107.0
3          59.0
5          56.0
6         104.0
7          53.0
10        122.0
11         82.0
14         31.0
16         88.0
17         86.0
18         26.0
19         92.0
21         95.0
22         38.0
23         22.0
24         34.0
25        101.0
29         40.0
31         42.0
32        196.0
34        148.0
36        104.0
39        120.0
41         23.0
44         38.0
45         43.0
47        201.0
48         61.0
50        201.0
          ...  
91529      48.0
91542      26.0
92259      27.0
94959      26.0
96079      24.0
97921      22.0
97938      22.0
98809      25.0
99114      44.0
102125     21.0
104841     29.0
106100     21.0
106487     26.0
106489     22.0
106782     29.0
106920     25.0
109374     35.0
109487     43.0
111362     24.0
111759     26.0
112552     28.0
112556     25.0
112852     37.0
115617     21.0
115713     26.0
116797     32.0
122882     31.0
122886     29.0
134130     25.0
134853     24.0
Name: rating, Le

### 1.2 Соединяем таблицы (1 балл)

Будет немного не удобно искать фильмы по их идентификаторам, нам нужны названия. Поэтому соедините две таблицы `ratings` и `movies` по `movieId` и отфильтруйте отзывы на редкие фильмы, которые мы нашли в предыдущем шаге. Выведите первые 5 элементов полученного датасета.

In [45]:
moviesMerge = ratings.merge(movies, left_on='movieId', right_index = True)
moviesMerge = moviesMerge[~moviesMerge['movieId'].isin(moviesRatingSorted)].dropna()
moviesMerge.head()

,userId,movieId,rating,timestamp,title,genres
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
3363,19,1029,5.0,855192033,Dumbo (1941),Animation|Children|Drama|Musical
6508,35,1029,1.5,1174450070,Dumbo (1941),Animation|Children|Drama|Musical
9226,61,1029,2.5,1216487645,Dumbo (1941),Animation|Children|Drama|Musical
10390,73,1029,3.0,1256030424,Dumbo (1941),Animation|Children|Drama|Musical


### 1.3 Матрица Фильмы-Пользователи

В нашем случае мы будем искать похожие фильмы, поэтому сделайте из датасета оценок матрицу, где в строках фильмы, в колонках пользователи, а в ячейках оценки. Пропущенные значение, то есть отсутсвие оценки, заполните 0.

Для увеличения скорости выполнения алгоритма ближайших соседей примените функцию [`scipy.sparse.csc_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html),  которая создаст плотное представление нашей разреженой матрицы.

In [46]:
from scipy.sparse import csr_matrix

moviesMerge.set_index(['userId', 'movieId'], inplace=True)
fu_matrix = csr_matrix((moviesMerge.rating, (moviesMerge.index.labels[1], moviesMerge.index.labels[0])))
fu_matrix

<8919x671 sparse matrix of type '<class 'numpy.float64'>'
	with 95433 stored elements in Compressed Sparse Row format>

### 1.4 Тренировка модели рекомендаций 

Так как по сути у нас нет каких-то меток классов, мы будем использовать алгоритма без учителя [`sklearn.neighbors.NearestNeighbors`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html). Научите его по данным нашей матрицы пользователей-фильмов и напишите **функцию**, принимающую на вход название фильма и возвращающую список из 6 рекомендованых фильмов, начиная от более похожего.

Для определения "похожести" двух фильмов мы будем использовать косинусную меру, вместо евклидова или минковского:

$$\LARGE cos(\overrightarrow{x},\overrightarrow{y}) = \frac{\overrightarrow{x} \cdot \overrightarrow{y}}{||\overrightarrow{x}|| \times ||\overrightarrow{y}||}$$

In [34]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric = 'cosine')
knn.fit(fu_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [86]:
def similar(name='Pulp Fiction (1994)', n=6):
    movieId = movies.title[movies.title == name].index.tolist()[0]
    dist, ind = knn.kneighbors(fu_matrix[movieId], n)
    similarMovies = [movies.iloc[movieId].title for movieId in ind[0]]
    
    return similarMovies

Выведите рекомендации для фильмов:
- `Pulp Fiction (1994)`
- `Star Wars: Episode V - The Empire Strikes Back (1980)`
- `Lord of the Rings: The Two Towers, The (2002)`

In [87]:
similar('Pulp Fiction (1994)')

['Fragile Trust: Plagiarism, Power, and Jayson Blair at the New York Times, A (2013)',
 'Imagine: John Lennon (1988)',
 'Three Ages (1923)',
 "Heaven's Prisoners (1996)",
 'Nell (1994)',
 'League of Extraordinary Gentlemen, The (a.k.a. LXG) (2003)']

In [88]:
similar('Star Wars: Episode V - The Empire Strikes Back (1980)')

['Underneath (1995)',
 'Virtuosity (1995)',
 'Walking Dead, The (1995)',
 'Vanya on 42nd Street (1994)',
 'Tom & Viv (1994)',
 'Luna de Avellaneda (2004)']

In [89]:
similar('Lord of the Rings: The Two Towers, The (2002)')

['Witless Protection (2008)',
 'Sweet Liberty (1986)',
 'Road to Wellville, The (1994)',
 'Smile Like Yours, A (1997)',
 'Silent Hill (2006)',
 'When a Stranger Calls (2006)']

### 1.5 Как посчитать качество рекомендации?

Как посчитать качество нашей классификации? У нас нет учителя, нет отложенной выборки. Мы просто посчитали все рейтинги и на основе них сделали предположения о похожести товаров. 

А как вы думаете можно проверить качество нашей рекомендательной системы? Напишите развернутый ответ своими словами.

**Ответ**: по моему мнению, проверить можно только на живых пользователях, которые смогут высказать свое "да" или "нет". На то несколько причин:

1. В матрице фильмы-пользователи очень много нулей (т.е. пользователь не ставил оценку)
2. Фильмов куда больше, чем пользователей
3. В данной "рекомендательной системе" мы не учитывае

## Наивный байесовский классификатор

На практике мы использовали его для детекции спам/не спам. В этот раз вы будете классифицировать [новостные тексты](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html). Попробуете разные методы  генерации признаков из текстов.

In [91]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### 2.1 Загрузка данных (2 балла)

Мы будем использовать только 4 класса текстов: `'alt.atheism', 'sci.space', 'talk.religion.misc', 'comp.graphics'`.

Загрузите тренировочную и тестовую части датасета для этих категорий.

In [92]:
categories = ['alt.atheism', 'sci.space', 'talk.religion.misc', 'comp.graphics']
train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', categories=categories, remove=('headers', 'footers', 'quotes'))

Выведите по 1 тексту для каждой из категории.

In [93]:
for id in range(len(categories)):
    print(train.data[id])
    print("========")

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych


Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries.

 >In article <1993Apr19.020359.26996@sq.sq.com>, msb@sq.sq.c

In [94]:
for id in range(len(categories)):
    print(test.data[id])
    print("========")

TRry the SKywatch project in  Arizona.
The Vatican library recently made a tour of the US.
 Can anyone help me in finding a FTP site where this collection is 
 available.
Hi there,

I am here looking for some help.

My friend is a interior decor designer. He is from Thailand. He is
trying to find some graphics software on PC. Any suggestion on which
software to buy,where to buy and how much it costs ? He likes the most
sophisticated 
software(the more features it has,the better)
RFD
                          Request For Discussion
                                for the
                          OPEN  TELEMATIC GROUP

                                  OTG

I have proposed the forming of a consortium/task force for the
promotion of NAPLPS/JPEG, FIF to openly discuss ways, method,
procedures,algorythms, applications, implementation, extensions of
NAPLPS/JPEG standards.  These standards should facilitate the creation
of REAL_TIME Online applications that make use of Voice, Video,
Telecomm

### 2.2 Мешок слов

Начнем с самого простого способа извлечения признаков из текстов: обычный подсчет количества употреблений слов в тексте.

Для этого используем [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Настройте его так, чтобы он использовал англиские стоп слова.

Проведите трансформаци текстов из обоих датасетов и выведите размер полученного словаря. (1 векторайзер для тестового и тренировочного датасета! У вас должны получиться матрицы с одинаковыми признаками-словами)

In [101]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
train_vector = vectorizer.fit_transform(train.data)
print(train_vector.shape)
test_vector = vectorizer.fit_transform(test.data)
print(test_vector.shape)

(2034, 26576)
(1353, 21240)


Будем использовать классифиактор для мультиклассифкации [sklearn.naive_bayes.MultinomialNB]().

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [105]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

cross_val_score(MultinomialNB(alpha=1), train_vector, cv=5).mean()

TypeError: fit() missing 1 required positional argument: 'y'

In [6]:
# Посмотрим на ТОП 10 по важности слова в каждой категории

def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: [%s]" % (category, " ".join(feature_names[top10])))

show_top10(clf, vec, newsgroups_train.target_names)

### 2.3 [Tf-IDF](https://ru.wikipedia.org/wiki/TF-IDF)

Мешок слов не учитывает "веса" слов, он просто смотрит их вхождение в документ. Вероятно, было бы полезно взвесить каким-то обраом каждое слово в документе. Действительно, если слово встречается во всех документах, то, наверное, его вес небольшой. А если редкое слово встречается в некоторых документах, то скорее всего оно какое-то узко тематическое.

Один из способов взвесить слова - это использовать меру tf-idf, где:

**TF - term frequency** - частота слова для каждой статьи

$$\LARGE \mathrm{tf}(t,d) = \frac{n_t}{\sum_k n_k}$$

**IDF - inverse document frequency*** — обратная частота документа - уменьшает вес часто встречаемых слов

$$\LARGE \mathrm{idf}(t, D) =  \log \frac{|D|}{|\{\,d_i \in D \mid t \in d_{i}\, \}|}$$

$D$ - число документов в корпусе

$|\{\,d_i \in D \mid t \in d_{i}\, \}|$ - число документов из коллекции ${\displaystyle D}$ , в которых встречается ${\displaystyle t}$  (когда ${\displaystyle n_{t}\neq 0}$ ).

**TF-IDF**

$$\LARGE \operatorname{tf-idf}(t,d,D) = \operatorname{tf}(t,d) \times \operatorname{idf}(t, D)$$


Для него есть класс [sklearn.feature_extraction.text.TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [8]:
show_top10(clf, vec, newsgroups_train.target_names)

### 2.4 В чём проблема?

Как вы заметили, качество немного улучшилось. Как вы думаете почему у нас до сих пор есть ошибки? В чём проблема? Опирайтесь на вывод топа 10 важных слов, confusion matrix и репорт классификации.

**Задание**: Напишите развернутое рассуждение.

Ответ: ???

![](https://i.imgur.com/3D5pgrk.png)

Можно еще немного улучшить качество предсказания, если использовть n-граммы, то есть не только одиночные слова, но и пары, тройки. Для этого у векторайзера есть параметр `ngram_range`, но будьте осторожны, сгенерируется очень много признаков. Их количество можно ограничить  `max_features`. Ещё можно копать в сторону правильного процессинга текстов: лемматизация и стемминг, чтобы убирать окончания, но это обчно не сильно помогает в английском языке.

Кроме того можно было бы пойти совсем в другую сторону и использовать word-2-vec технологии, тогда бы у нас слова начал "обретать" смысл. Но это вовсем другая история.